# PFRA 

### _Development Notebook_
--- 

### QA/QC
#### Global Errors Check

This notebook is intended to identify global issues in a model simulation that require follow-up responses from the modeler before the model can be used in *STARRII* PFRA studies.

Global issues currently implemented include:
 
 __1. Check Model inputs__
 
    A. Plan Information
    B. Plan Parameters
    C. Boundary Conditions 
     
__2. Check Results for each domain (2D Flow Area) in the model__
 
    A. Computational Summary & Depth Plots
    B. Domain Extents
    C. High Velocities at cell faces
 
 
Global issues *in development* include:
 - Mapping areas where Courant number exceeds indicated threshold.
 - Mapping areas where iteration counts exceed indicated threshold.

In [10]:
model_s3path = "s3://azavea/jwx_test/data/zip/DC_F02_NBR_E0029_out.zip"

In [11]:
import sys;sys.path.append('../')

In [12]:
import gdal
from hecrasio.core import *
from hecrasio.qaqc import *
import scrapbook as sb

#### Read model output data 

*from AWS*

In [13]:
model = ResultsZip(model_s3path, require_prj=False)
hdfResults_path = pull_result_paths(model)

In [14]:
rasPlan = HDFResultsFile(model, model_s3path, hdfResults_path)

AttributeError: 'HDFResultsFile' object has no attribute '_HDFResultsFile__path'

### 1A. Plan Information

In [ ]:
sb.glue(rasPlan.Plan_Information.to_json(), "json")
rasPlan.Plan_Information

### 1B. Plan Parameters

In [ ]:
sb.glue(rasPlan.Plan_Parameters.to_json(), "json")
rasPlan.Plan_Parameters

### 1C. Boundary Conditions 

#### Input hydrographs

In [ ]:
results_data = show_results(rasPlan.domains, model, rasPlan)
sb.glue(results_data.to_json(), "json")

### 2A. Computational Summary

In [ ]:
non_nan = rasPlan.summary.replace(np.nan, "00:00:00")
sb.glue(non_nan.to_json(), "json")
rasPlan.summary

# END